# Dynamic-DeepHit Tutorial

### by Changhee Lee

In [1]:
!pip install lifelines 
!pip install -U numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 3.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=343dfcb066d95cf1660d3e3f8bf9b7291d0b3109ab1b49c48c62c204e0c78b0f
  Stored in directory: /root/.cache/pip/wheels/16/a2/b6/582cfdfbeeccd469504a01af3bb952fd9e7eccba40995eafea
Successfully built autograd-gamma
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 45.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into 

In [5]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/Longitudinal Risk Prediction/Dynamic-DeepHit LRPP')

%tensorflow_version 1.x

Mounted at /content/drive


ValueError: ignored

In [4]:
_EPSILON = 1e-08

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

import matplotlib.pyplot as plt

import import_data as impt
#import AUC_calculator

from class_DeepLongitudinal import Model_Longitudinal_Attention

from utils_eval             import c_index, brier_score
from utils_log              import save_logging, load_logging
from utils_helper           import f_get_minibatch, f_get_boosted_trainset

warnings.filterwarnings('ignore')

import importlib

importlib.reload(impt)

ModuleNotFoundError: ignored

In [ ]:
!nvidia-smi
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
def _f_get_pred(sess, model, data, data_mi, pred_horizon):
    '''
        predictions based on the prediction time.
        create new_data and new_mask2 that are available previous or equal to the prediction time (no future measurements are used)
    '''
    new_data    = np.zeros(np.shape(data))
    new_data_mi = np.zeros(np.shape(data_mi))

    meas_time = np.concatenate([np.zeros([np.shape(data)[0], 1]), np.cumsum(data[:, :, 0], axis=1)[:, :-1]], axis=1)

    for i in range(np.shape(data)[0]):
        last_meas = np.sum(meas_time[i, :] <= pred_horizon)

        new_data[i, :last_meas, :]    = data[i, :last_meas, :]
        new_data_mi[i, :last_meas, :] = data_mi[i, :last_meas, :]

    return model.predict(new_data, new_data_mi)


def f_get_risk_predictions(sess, model, data_, data_mi_, pred_time, eval_time):
    
    pred = _f_get_pred(sess, model, data_[[0]], data_mi_[[0]], 0)
    _, num_Event, num_Category = np.shape(pred)
       
    risk_all = {}
    for k in range(num_Event):
        risk_all[k] = np.zeros([np.shape(data_)[0], len(pred_time), len(eval_time)])
            
    for p, p_time in enumerate(pred_time):
        ### PREDICTION
        pred_horizon = int(p_time)
        pred = _f_get_pred(sess, model, data_, data_mi_, pred_horizon)


        for t, t_time in enumerate(eval_time):
            eval_horizon = int(t_time) + pred_horizon #if eval_horizon >= num_Category, output the maximum...

            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
            risk = np.sum(pred[:,:,pred_horizon:(eval_horizon+1)], axis=2) #risk score until eval_time
            risk = risk / (np.sum(np.sum(pred[:,:,pred_horizon:], axis=2), axis=1, keepdims=True) +_EPSILON) #conditioniong on t > t_pred
            
            for k in range(num_Event):
                risk_all[k][:, p, t] = risk[:, k]
                
    return risk_all

### 1. Import Dataset
#####      - Users must prepare dataset in csv format and modify 'import_data.py' following our examplar 'PBC2'

In [ ]:
data_mode                   = 'PBC2' 
seed                        = 1234

##### IMPORT DATASET
'''
    num_Category            = max event/censoring time * 1.2
    num_Event               = number of evetns i.e. len(np.unique(label))-1
    max_length              = maximum number of measurements
    x_dim                   = data dimension including delta (1 + num_features)
    x_dim_cont              = dim of continuous features
    x_dim_bin               = dim of binary features
    mask1, mask2, mask3     = used for cause-specific network (FCNet structure)
'''

if data_mode == 'PBC2':
    (x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi), df_raw = impt.import_dataset(norm_mode = 'standard')
    
    
    # This must be changed depending on the datasets, prediction/evaliation times of interest
    pred_time = [8] # prediction time (in years)
    eval_time = [10] # years evaluation time (for C-index and Brier-Score)
else:
    print ('ERROR:  DATA_MODE NOT FOUND !!!')

_, num_Event, num_Category  = np.shape(mask1)  # dim of mask3: [subj, Num_Event, Num_Category]
max_length                  = np.shape(data)[1]


file_path = '{}'.format(data_mode)

if not os.path.exists(file_path):
    os.makedirs(file_path)

### 2. Set Hyper-Parameters
##### - Play with your own hyper-parameters!

In [ ]:
burn_in_mode                = 'OFF' #{'ON', 'OFF'}
boost_mode                  = 'OFF' #{'ON', 'OFF'}

##### HYPER-PARAMETERS
new_parser = {'mb_size': 128,

             'iteration_burn_in': 3000,
             'iteration': 10000,

             'keep_prob': 1,
             'lr_train': 5e-3,

             'h_dim_RNN': 50,
             'h_dim_FC' : 100,
             'num_layers_RNN':5,
             'num_layers_ATT':2,
             'num_layers_CS' :4,

             'RNN_type':'LSTM', #{'LSTM', 'GRU'}

             'FC_active_fn' : tf.nn.relu,
             'RNN_active_fn': tf.nn.tanh,

            'reg_W'         : 1e-05,
            'reg_W_out'     : 1e-05,

             'alpha' :2.0,
             'beta'  :0.0,
             'gamma' :1.0
}


# INPUT DIMENSIONS
input_dims                  = { 'x_dim'         : x_dim,
                                'x_dim_cont'    : x_dim_cont,
                                'x_dim_bin'     : x_dim_bin,
                                'num_Event'     : num_Event,
                                'num_Category'  : num_Category,
                                'max_length'    : max_length }

# NETWORK HYPER-PARMETERS
network_settings            = { 'h_dim_RNN'         : new_parser['h_dim_RNN'],
                                'h_dim_FC'          : new_parser['h_dim_FC'],
                                'num_layers_RNN'    : new_parser['num_layers_RNN'],
                                'num_layers_ATT'    : new_parser['num_layers_ATT'],
                                'num_layers_CS'     : new_parser['num_layers_CS'],
                                'RNN_type'          : new_parser['RNN_type'],
                                'FC_active_fn'      : new_parser['FC_active_fn'],
                                'RNN_active_fn'     : new_parser['RNN_active_fn'],
                                'initial_W'         : tf.contrib.layers.xavier_initializer(),

                                'reg_W'             : new_parser['reg_W'],
                                'reg_W_out'         : new_parser['reg_W_out']
                                 }


mb_size           = new_parser['mb_size']
iteration         = new_parser['iteration']
iteration_burn_in = new_parser['iteration_burn_in']

keep_prob         = new_parser['keep_prob']
lr_train          = new_parser['lr_train']

alpha             = new_parser['alpha']
beta              = new_parser['beta']
gamma             = new_parser['gamma']

# SAVE HYPERPARAMETERS
log_name = file_path + '/hyperparameters_log.txt'
save_logging(new_parser, log_name)

### 3. Split Dataset into Train/Valid/Test Sets

In [ ]:
indecies_mesa = pd.read_csv('/content/drive/My Drive/Longitudinal Risk Prediction/Dynamic-DeepHit LRPP/random_index_select.csv')
indecies_mesa = np.array(indecies_mesa['study_index'])

df_index = list(range(len(data)))

def train_test_split_index(df, indecies):
    tr_data = df[indecies == False]
    te_data = df[indecies]
    
    return tr_data, te_data

### List Split Option
split_option = 'MESA'

if split_option == 'MESA':

    ### Select MESA as Testing set 
    tr_data, te_data = train_test_split_index(data, indecies_mesa)
    tr_data_raw, te_data_raw = train_test_split_index(df_raw, indecies_mesa)
    tr_df_index, te_df_index = train_test_split_index(np.array(df_index), indecies_mesa)
    tr_data_mi, te_data_mi = train_test_split_index(data_mi, indecies_mesa)
    tr_time, te_time = train_test_split_index(time, indecies_mesa)
    tr_label, te_label = train_test_split_index(label, indecies_mesa)
    tr_mask1, te_mask1 = train_test_split_index(mask1, indecies_mesa)
    tr_mask2, te_mask2 = train_test_split_index(mask2, indecies_mesa)
    tr_mask3, te_mask3 = train_test_split_index(mask3, indecies_mesa) 
else:
    ### TRAINING-TESTING SPLIT
    (tr_data, te_data, tr_data_raw, te_data_raw, tr_df_index, te_df_index, tr_data_mi, te_data_mi, tr_time, te_time, tr_label, te_label, 
    tr_mask1,te_mask1, tr_mask2,te_mask2, tr_mask3,te_mask3) = train_test_split(data, df_raw, df_index, data_mi, time, label, mask1, mask2, mask3, test_size=0.2, random_state=seed, stratify = label) 

(tr_data, va_data, tr_data_mi, va_data_mi, tr_time, va_time, tr_label, va_label, 
 tr_mask1,va_mask1, tr_mask2,va_mask2, tr_mask3,va_mask3) = train_test_split(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3, test_size= 0.25, random_state=seed, stratify = tr_label) 

if boost_mode == 'ON':
    tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)

In [ ]:
# get validation AUCs
validation_data = True
if validation_data == True:
  te_data = va_data
  te_time = va_time
  te_label = va_label
  te_mask1 = va_mask1
  te_mask2 = va_mask2
  te_mask3 = va_mask3

training_data = False
if training_data == True:
  te_data = tr_data
  te_time = tr_time
  te_label = tr_label
  te_mask1 = tr_mask1
  te_mask2 = tr_mask2
  te_mask3 = tr_mask3

In [ ]:
##### CREATE DYNAMIC-DEEPHIT NETWORK
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())

In [ ]:
### TRAINING - BURN-IN
if burn_in_mode == 'ON':
    print( "BURN-IN TRAINING ...")
    for itr in range(iteration_burn_in):
        x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
        DATA = (x_mb, k_mb, t_mb)
        MISSING = (x_mi_mb)

        _, loss_curr = model.train_burn_in(DATA, MISSING, keep_prob, lr_train)

        if (itr+1)%1000 == 0:
            print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))


### TRAINING - MAIN
print( "MAIN TRAINING ...")
min_valid = 0.5

for itr in range(iteration):
    x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
    DATA = (x_mb, k_mb, t_mb)
    MASK = (m1_mb, m2_mb, m3_mb)
    MISSING = (x_mi_mb)
    PARAMETERS = (alpha, beta, gamma)

    _, loss_curr = model.train(DATA, MASK, MISSING, PARAMETERS, keep_prob, lr_train)

    if (itr+1)%1000 == 0:
        print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))

    ### VALIDATION  (based on average C-index of our interest)
    if (itr+1)%1000 == 0:        
        risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)
        
        for p, p_time in enumerate(pred_time):
            pred_horizon = int(p_time)
            val_result1 = np.zeros([num_Event, len(eval_time)])
            
            for t, t_time in enumerate(eval_time):                
                eval_horizon = int(t_time) + pred_horizon
                for k in range(num_Event):
                    val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            
            if p == 0:
                val_final1 = val_result1
            else:
                val_final1 = np.append(val_final1, val_result1, axis=0)

        tmp_valid = np.mean(val_final1)

        if tmp_valid >  min_valid:
            min_valid = tmp_valid
            saver.save(sess, file_path + '/model')
            print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))

### 5. Test the Trained Network

In [ ]:
saver.restore(sess, file_path + '/model')
#saver.restore(sess, '/content/drive/My Drive/Longitudinal Risk Prediction/Dynamic-DeepHit LRPP/PBC2/model')

risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

for p, p_time in enumerate(pred_time):
    pred_horizon = int(p_time)
    result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

    for t, t_time in enumerate(eval_time):                
        eval_horizon = int(t_time) + pred_horizon
        for k in range(num_Event):
            result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
   
    if p == 0:
        final1, final2 = result1, result2
    else:
        final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)
        
        
row_header = []
for p_time in pred_time:
    for k in range(num_Event):
        row_header.append('pred_time {}: event_{}'.format(p_time,k+1))
            
col_header = []
for t_time in eval_time:
    col_header.append('eval_time {}'.format(t_time))

# c-index result
df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

# brier-score result
df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

### PRINT RESULTS
print('========================================================')
print('--------------------------------------------------------')
print('- C-INDEX: ')
print(df1)
print('--------------------------------------------------------')
print('- BRIER-SCORE: ')
print(df2)
print('========================================================')

### AUC Calculation

In [ ]:
def prediction_df(risk_all, event):
    """
    Gets the individual predictions from risk_all
    event = 1
    """
    risk_list = risk_all[event-1].tolist()

    risk_list = [risk for risk_sublist in risk_list for risk in risk_sublist]
    risk_list = [risk for risk_sublist in risk_list for risk in risk_sublist]

    return risk_list

def prediction_pred_values(pred_t, eval_t , event_n, risk):
    """Gets the predication values and true labels for AUC CI calculation"""
    
    global true_list
    
    time_horizon = pred_t + eval_t 
    true = (te_time <= time_horizon) * (te_label == event_n).astype(int).tolist()
    
    true_list = [true_i for true_sublist in true for true_i in true_sublist]
    true_list = [true_i for true_sublist in true for true_i in true_sublist]

    pred_df = pd.DataFrame({'ddh_risk' : risk, 
                            'true_labels' : true_list})
    
    return pred_df

def prediction_ROC(pred_t, eval_t , event_n, df):
    """Calculates the AUC at each prediction time and prediction time"""
    
    global true 
    
    time_horizon = pred_t + eval_t 
    true = (te_time <= 15) * (te_label == 1).astype(int)

    fpr, tpr, thresh = roc_curve(true.tolist(), df.loc[(df.pred_time == pred_t) & (df.eval_time == eval_t), 'value'].tolist())
            
    return fpr, tpr, thresh

In [ ]:
# Get individual risks for AUC calculation (DDH)
risk_df = prediction_df(risk_all, 1)
pred_df = prediction_pred_values(8, 10, 1, risk_df)
round(roc_auc_score(pred_df['true_labels'], pred_df['ddh_risk']), 3)

In [ ]:
pred_df.to_csv('/content/drive/My Drive/Longitudinal Risk Prediction/Dynamic-DeepHit LRPP/pred_df_gender.csv', index = False)

In [ ]:
 pred_df.ddh_risk.hist()

In [ ]:
# output risk 
#pred_df.to_csv('/content/drive/My Drive/Longitudinal Risk Prediction/Dynamic-DeepHit LRPP/DDH_pred_df_current.csv', index = False)